# Type annotations

Python has an option to annotate types, which is useful during relatively serious development.

Check resources:

- [Python type checking](https://realpython.com/python-type-checking/#type-systems) article on real python.
- [PEP 484](https://peps.python.org/pep-0484/) that introduce type hints.
- [PEP 526](https://peps.python.org/pep-0526/) that introduce variable annotations.

In [1]:
from typeguard import typechecked

## Usecases

### Functions annotations

There is an option to annotate the function. You can annotate type for arguments and type for output value. For more details check [specific page](type_annotations/function_annotations.ipynb).

In [4]:
@typechecked
def some_function(arg1: float, arg2: int) -> bool:
    return arg1 > arg2

some_function(4., 2)

TypeError: <module '__main__'> is a built-in module

### Variables annotations

You can annotate the type of variable in your code.

In [14]:
pi: float = 3.142

## Annotation cases

### Complex types annotation

There is a special syntax for annotating the types of elements of complex types. Find out more about annotations for complex data types at [specific page](type_annotations/complex_types.ipynb).

Here is a short example of annotations for different complex types.

In [8]:
my_tuple: tuple[int, float, bool] = (10, 4., True)
my_list: list[int] = [10,20,30]
my_dict: dict[str, float] = {"item1" : 3., "item2" : 7.}
my_set: set[int] = {3,2,1,4}

### Any type

Some times you'll meet cases where object can take any type. In most cases you can just ignore typing for it. But there are reasons why you should have option to declare that expression can have any type:

- To show that any type is a deliberate decision.
- To have an option for cases where a type must be specified, such as the type of keys in a dictionary or the type of a particular element in a tuple.

Consider the example where there are functions that return the key of the item with the maximum value from dict. But keys can be of any type that is acceptable to be keys for dictionaries. So it can be completed with syntax below.

In [13]:
from typing import Any
def max_key(inp_dict: dict[Any, int|float]) -> Any:
    return max(inp_dict, key=inp_dict.get)

max_key({10: 3, "hello": 7})

'hello'

## Type aliases

Note that you can save your annotations as regular python object - your programmes will be neater.

Below is an example that defines a function to create triangles as a tuple of three two-dimensional points. To make things simplier, you can define an annotation for the point and use it to define an annotation for the triangle.

In [5]:
from random import random

# Here is straighforward definition
def get_triangle() -> tuple[
        tuple[float, float],
        tuple[float, float],
        tuple[float, float]
    ]:
    return (
        (random(), random())
        for i in range(3)
    )

# Same but shorter
point = tuple[float, float]
def get_triangle() -> tuple[point, point, point]:
    return (
        (random(), random())
        for i in range(3)
    )

## `__annotations__` attribute

The `__annotations__` attribute allows you to retrieve annotated types for a Python object.

For function it returns a dictionary with keys corresponding to the names of the arguments and values corresponding to the types of the arguments. The return type of the function can be accessed using the `return` key.

In [1]:
def some_function(arg1: float, arg2: bool) -> int:
    if arg1 > 10 and arg2:
        return 20

some_function.__annotations__

{'arg1': float, 'arg2': bool, 'return': int}

You can simply access it from any Python namespace to get annotations of the waribles in that namespace.

In [2]:
new_variable : float = 10
test_variable : float = 10

__annotations__

{'new_variable': float, 'test_variable': float}

## Mypy type checker

Tool that allows you to check if the programme respects the specified type hints.

So in the following example a function is defined that takes two arguments and they are annotated as `int`. And just in the next line funciton call just ignores annotations.

In [8]:
%%writefile type_annotations_files/mypy_example.py
def bin_sum(a: int, b: int):
    return a+b

print(bin_sum("a", "b"))

Overwriting type_annotations_files/mypy_example.py


Just execution of the program won't cause any problems.

In [9]:
!python3 type_annotations_files/mypy_example.py

ab


Which strings do not meet the type annotation.

In [10]:
!python3 -m mypy type_annotations_files/mypy_example.py

type_annotations_files/mypy_example.py:4: error: Argument 1 to "bin_sum" has incompatible type "str"; expected "int"  [arg-type]
type_annotations_files/mypy_example.py:4: error: Argument 2 to "bin_sum" has incompatible type "str"; expected "int"  [arg-type]
Found 2 errors in 1 file (checked 1 source file)


### Get mypy types

`Mypy` has it's own type detection system, so sometimes it's useful to know how to check what type it's detecting. You can do this using the `reveal_type` and `reveal_locals` functions.

**Note** You don't need to import them or something `mypy` will understand them by itself.

Here is an example of the `reveal_type` function, which allows you to understand how mypy interprets the type of a given name.

In [8]:
%%writefile type_annotations_files/mypy_reveal_type.py
import numpy as np
reveal_type(np.pi)

Writing type_annotations_files/mypy_reveal_type.py


In [10]:
!python3 -m mypy type_annotations_files/mypy_reveal_type.py

type_annotations_files/mypy_reveal_type.py:2: note: Revealed type is "builtins.float"
Success: no issues found in 1 source file


And an example of `reveal_locals`, which prints out information about all the variables in the namespace.

In [11]:
%%writefile type_annotations_files/mypy_reveal_locals.py
my_float = 10.
my_int = 10

reveal_locals()

Writing type_annotations_files/mypy_reveal_locals.py


In [12]:
!python3 -m mypy type_annotations_files/mypy_reveal_locals.py

type_annotations_files/mypy_reveal_locals.py:4: note: Revealed local types are:
type_annotations_files/mypy_reveal_locals.py:4: note:     my_float: builtins.float
type_annotations_files/mypy_reveal_locals.py:4: note:     my_int: builtins.int
Success: no issues found in 1 source file


### Wrong syntax

The Python interpreter doesn't throw exceptions even if there are annotations that are defined incorrectly. But `mypy` will throw errors on it's side.

Here is an example where we are trying to annotate the dictionary, but instead of annotating key and value types, we have only annotated one.

In [7]:
%%writefile type_annotations_files/mypy_wrong_syntax.py
my_dict: dict[int]

Writing type_annotations_files/mypy_wrong_syntax.py


If you try to run it just with the python interpreter, it works fine.

In [8]:
!python3 type_annotations_files/mypy_wrong_syntax.py

But `mypy` raises corresponding error.

In [9]:
!python3 -m mypy type_annotations_files/mypy_wrong_syntax.py

type_annotations_files/mypy_wrong_syntax.py:1: error: "dict" expects 2 type arguments, but 1 given  [type-arg]
Found 1 error in 1 file (checked 1 source file)
